## 2. การทำความสะอาดข้อมูล หรือคัดกรองข้อมูล

จากขั้นตอนที่แล้ว เริ่มจากดูชนิดของข้อมูลก่อน

In [38]:
import pandas as pd

In [39]:
data_hourly = pd.read_csv("../data/stations_hourly.csv")

In [40]:
describe_col = data_hourly.columns[:11] #column ที่ต้องการ
data_hourly = data_hourly[describe_col]

In [41]:
data_hourly.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5951903 entries, 0 to 5951902
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   datetime    5951903 non-null  object 
 1   station_id  5951903 non-null  int64  
 2   PM2.5       4051918 non-null  float64
 3   PM10        4637964 non-null  float64
 4   NOx         3046803 non-null  float64
 5   O3          4914802 non-null  float64
 6   CO          4542033 non-null  float64
 7   HR          3388063 non-null  float64
 8   NO          3134901 non-null  float64
 9   NO2         4344013 non-null  float64
 10  TMP         3467518 non-null  float64
dtypes: float64(9), int64(1), object(1)
memory usage: 499.5+ MB


สังเกต column ที่ datetime ไม่เป็น datetime type จำเป็นต้องแปลง Type  
โดย มีข้อมูลใน stations_rsinaica.csv ที่เกี่ยวกับ Timezone

In [42]:
station_data = pd.read_csv("../data/stations_rsinaica.csv")

ยกตัวอย่างข้อมูลสถานี id = 32

In [43]:
station_data.loc[station_data["station_id"] == 32]["timezone"]

17    Tiempo del centro, UTC-6 (UTC-5 en verano)
Name: timezone, dtype: object

จะมีข้อมูล Timezone อยู่ ซึ่งสรุปได้ดังนี้ **(เทียบกับ pytz library)**  
  
> UTC-6 = centro  
> : America/Mexico_City  
> UTC-7 = pacífico, Sonora  
> : America/Chihuahua  
> UTC-8 = noroeste  
> : America/Tijuana  
  
หมายเหตุ centro, pacífico, Sonora, noroeste เป็นคำ spain ที่บันทึกใน database column timezone  

เริ่มการแปลง datetime (ทำเฉพาะ hourly)  
(ตั้ง timezone และ แปลงเป็น UTC0)

In [44]:
import numpy as np

In [45]:
ListTimeZone = {"centro" : "America/Mexico_City", "pacífico" : "America/Chihuahua",\
                "Sonora" : "America/Chihuahua" ,"noroeste" : "America/Tijuana"}
KeysTimeZone = list(ListTimeZone.keys())

In [46]:
data_group = data_hourly.groupby("station_id")
key_group = data_group.groups

In [47]:
for id in key_group:
    station_group = data_group.get_group(id)
    row_station = station_data.loc[ station_data["station_id"] == id ]
    str_timezone = row_station["timezone"].values[0]
    timezone = ""
    
    for timekey in KeysTimeZone:
        if timekey in str_timezone:
            timezone = ListTimeZone[timekey]
            break
    
    infer_dst = np.array([False] * station_group.shape[0]) # from stackoverflow
    
    dateconv = pd.to_datetime(station_group["datetime"])
    dateconv = dateconv.dt.tz_localize(timezone, nonexistent='shift_forward', ambiguous=infer_dst)
    dateconv = dateconv.dt.tz_convert("UTC")
    
    data_hourly.loc[station_group.index, ("datetime")] = dateconv

ผลลัพธ์หลังแปลง

In [48]:
data_hourly["datetime"]

0          2015-04-02 15:00:00+00:00
1          2015-05-21 23:00:00+00:00
2          2015-05-22 00:00:00+00:00
3          2015-05-22 00:00:00+00:00
4          2015-05-22 01:00:00+00:00
                     ...            
5951898    2022-01-01 01:00:00+00:00
5951899    2022-01-01 02:00:00+00:00
5951900    2022-01-01 03:00:00+00:00
5951901    2022-01-01 04:00:00+00:00
5951902    2022-01-01 05:00:00+00:00
Name: datetime, Length: 5951903, dtype: object

ด้วยความที่มีข้อมูลบางส่วนเก็บก่อนปี 2010 (ข้อมูลที่ต้องการคือปี 2010-2021) จึงต้องตัดออก

In [49]:
data_hourly.drop(data_hourly[data_hourly["datetime"] < pd.Timestamp(year=2010, month=1, day=1, tz="UTC")].index, inplace=True)

In [50]:
data_hourly.index = range(0, data_hourly.shape[0])

In [51]:
data_hourly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5848182 entries, 0 to 5848181
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   datetime    object 
 1   station_id  int64  
 2   PM2.5       float64
 3   PM10        float64
 4   NOx         float64
 5   O3          float64
 6   CO          float64
 7   HR          float64
 8   NO          float64
 9   NO2         float64
 10  TMP         float64
dtypes: float64(9), int64(1), object(1)
memory usage: 490.8+ MB


ต่อมา ตัดค่า outliner

describe ก่อนจะตัดค่า

In [52]:
data_hourly.describe()

,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,TMP
count,5.848182e+06,4.051918e+06,4.637964e+06,3.046803e+06,4.914802e+06,4.542033e+06,3.388063e+06,3.031180e+06,4.344013e+06,3.467518e+06
mean,1.643294e+02,1.575017e+33,7.423936e+32,3.370523e+00,2.235814e-01,4.293768e+00,2.091774e+02,3.266586e+00,2.309685e+00,2.084149e+01
std,8.867297e+01,8.354006e+35,4.069897e+35,3.181981e+03,1.525898e+02,4.671265e+02,2.299328e+05,3.190170e+03,2.664859e+03,6.214520e+02
min,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.020000e+02,1.100000e+01,2.536100e+01,1.141000e-02,8.000000e-03,4.300000e-01,4.156241e+01,3.000000e-03,8.000000e-03,1.210000e+01
50%,1.370000e+02,1.800000e+01,4.108353e+01,2.200000e-02,1.970710e-02,7.200000e-01,6.297000e+01,7.000000e-03,1.520973e-02,1.763626e+01
75%,2.450000e+02,2.900000e+01,6.500000e+01,4.276215e-02,3.579000e-02,1.270000e+00,7.900000e+01,2.090000e-02,2.500000e-02,2.309000e+01
max,4.260000e+02,4.924800e+38,2.640400e+38,3.422585e+06,2.023916e+05,6.947900e+05,4.166700e+08,3.422585e+06,3.422585e+06,4.208200e+05


จำนวนช่องข้อมูลที่ว่างก่อนตัดค่า

In [53]:
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         1796264
PM10          1210218
NOx           2801379
O3             933380
CO            1306149
HR            2460119
NO            2817002
NO2           1504169
TMP           2380664
dtype: int64

หาจุดตัดค่าเกินเกณฑ์ โดยใช้ เปอร์เซ็นไทล์ที่ 98.5 และ 5

In [54]:
high_all_cut = data_hourly.quantile(q=0.985, numeric_only=True)
low_all_cut = data_hourly.quantile(q=0.05, numeric_only=True)

In [55]:
for col in describe_col[2:]:
    data_cut_all = data_hourly.loc[(data_hourly[col] > low_all_cut[col]) & (data_hourly[col] < high_all_cut[col])]
    data_hourly.loc[:, (col)] = [np.nan] * data_hourly.shape[0]
    data_hourly.loc[data_cut_all.index, (col)] = data_cut_all

describe และ จำนวนช่องว่าง หลังตัดค่าเกินเกณฑ์

In [56]:
data_hourly.describe()

,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,TMP
count,5.848182e+06,3.787115e+06,4.336441e+06,2.846175e+06,4.533204e+06,4.244356e+06,3.163200e+06,2.832746e+06,4.059010e+06,3.239513e+06
mean,1.643294e+02,2.413626e+01,5.299561e+01,3.442132e-02,2.534591e-02,9.888091e-01,6.147723e+01,1.836882e-02,1.856115e-02,1.819221e+01
std,8.867297e+01,2.503097e+01,3.988911e+01,3.400579e-02,1.926691e-02,7.311852e-01,2.108099e+01,2.680735e-02,1.211658e-02,6.524575e+00
min,3.200000e+01,3.012900e+00,9.400056e+00,3.200100e-03,2.000081e-03,1.560028e-01,1.663104e+01,5.000100e-04,2.100028e-03,6.070100e+00
25%,1.020000e+02,1.200000e+01,2.783593e+01,1.249000e-02,9.246800e-03,4.850000e-01,4.488734e+01,3.400000e-03,9.000000e-03,1.283000e+01
50%,1.370000e+02,1.900000e+01,4.300000e+01,2.300000e-02,2.100000e-02,7.550000e-01,6.400000e+01,7.484900e-03,1.600000e-02,1.800000e+01
75%,2.450000e+02,2.900000e+01,6.500000e+01,4.300000e-02,3.600000e-02,1.280000e+00,7.900000e+01,2.100000e-02,2.507895e-02,2.310000e+01
max,4.260000e+02,3.018800e+02,3.270700e+02,2.069858e-01,9.399300e-02,4.166400e+00,9.999999e+01,1.729900e-01,6.339660e-02,3.361900e+01


In [57]:
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         2061067
PM10          1511741
NOx           3002007
O3            1314978
CO            1603826
HR            2684982
NO            3015436
NO2           1789172
TMP           2608669
dtype: int64

ต่อมา เติมค่าที่ว่างด้วยค่าเฉลี่ย

ฟังก์ชันที่ใช้

In [58]:
def fillna_avg(freq_time, data_input):
    data_group_date = data_input.groupby([pd.Grouper(freq=freq_time, key="datetime"), "station_id"], dropna=True)
    mean_all = data_group_date.mean()
    date_key = list(data_group_date.groups.keys())
    
    for date_id in date_key:
        try:
            data_select = data_group_date.get_group(date_id)
            avg = mean_all.loc[date_id]
            #print(avg)
            for col in describe_col[2:]:
                avg_value = avg[col]
                if str(avg_value) != "nan":
                    data_input.loc[data_select.index, (col)] = data_input.loc[data_select.index, (col)].fillna(avg_value)
        except KeyError:
            continue
    
    return data_input

ช่องว่างข้อมูลก่อนเติมค่า

In [59]:
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         2061067
PM10          1511741
NOx           3002007
O3            1314978
CO            1603826
HR            2684982
NO            3015436
NO2           1789172
TMP           2608669
dtype: int64

หลังจากเติม โดยใช้ค่าเฉลี่ยรายวัน

In [60]:
data_hourly = fillna_avg("1D", data_hourly)
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         2050486
PM10          1489855
NOx           2990637
O3            1310018
CO            1579404
HR            2678057
NO            3003229
NO2           1762222
TMP           2508589
dtype: int64

หลังจากเติม โดยใช้ค่าเฉลี่ยรายสัปดาห์

In [61]:
data_hourly = fillna_avg("7D", data_hourly)
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         2048050
PM10          1488546
NOx           2988751
O3            1308111
CO            1577544
HR            2676520
NO            3001259
NO2           1759160
TMP           2507648
dtype: int64

หลังจากเติม โดยใช้ค่าเฉลี่ยราย 2 สัปดาห์

In [62]:
data_hourly = fillna_avg("14D", data_hourly)
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         2046117
PM10          1487574
NOx           2986345
O3            1306370
CO            1575653
HR            2675118
NO            2999195
NO2           1757102
TMP           2506661
dtype: int64

หลังจากเติม โดยใช้ค่าเฉลี่ยรายเดือน

In [63]:
data_hourly = fillna_avg("1M", data_hourly)
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         2038770
PM10          1484437
NOx           2982835
O3            1303912
CO            1569249
HR            2671672
NO            2993870
NO2           1752825
TMP           2503766
dtype: int64

หลังจากเติม โดยใช้ค่าเฉลี่ยราย 3 เดือน

In [64]:
data_hourly = fillna_avg("3M", data_hourly)
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         2016927
PM10          1474678
NOx           2970012
O3            1292862
CO            1555640
HR            2664889
NO            2975286
NO2           1738261
TMP           2495873
dtype: int64

หลังจากเติม โดยใช้ค่าเฉลี่ยราย 6 เดือน

In [65]:
data_hourly = fillna_avg("6M", data_hourly)
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         1981263
PM10          1455734
NOx           2946201
O3            1274987
CO            1528118
HR            2642102
NO            2938613
NO2           1713126
TMP           2474257
dtype: int64

หลังจากเติม โดยใช้ค่าเฉลี่ยรายปี

In [66]:
data_hourly = fillna_avg("1Y", data_hourly)
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         1914244
PM10          1369826
NOx           2884149
O3            1200376
CO            1471581
HR            2597930
NO            2845786
NO2           1657861
TMP           2435255
dtype: int64

หลังจากเติม โดยใช้ค่าเฉลี่ยราย 3 ปี

In [67]:
data_hourly = fillna_avg("3Y", data_hourly)
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         1573434
PM10          1219012
NOx           2584692
O3             982316
CO            1219207
HR            2415329
NO            2381526
NO2           1374978
TMP           2226485
dtype: int64

หลังจากเติม โดยใช้ค่าเฉลี่ยราย 6 ปี

In [68]:
data_hourly = fillna_avg("6Y", data_hourly)
data_hourly.isna().sum()

datetime            0
station_id          0
PM2.5         1112282
PM10           964166
NOx           1927146
O3             710449
CO             873012
HR            1637981
NO            1776954
NO2            974572
TMP           1547230
dtype: int64

หลังจากเติม โดยใช้ค่าเฉลี่ยราย 11 ปี (แทบทั้งหมด)

In [69]:
data_hourly = fillna_avg("11Y", data_hourly)
data_hourly.isna().sum()

datetime           0
station_id         0
PM2.5         251524
PM10          577548
NOx           595181
O3            294199
CO            394821
HR            602107
NO            570415
NO2           399948
TMP           558123
dtype: int64

เนื่องจากยังเหลือช่องว่างอยู่ จึงเติมด้วยค่าเฉลี่ยของทั้งหมด

In [70]:
mean_all = data_hourly[describe_col[2:]].mean()

for col in describe_col[2:]:
    data_hourly.loc[:, (col)].fillna(mean_all[col], inplace=True)

In [71]:
data_hourly.isna().sum()

datetime      0
station_id    0
PM2.5         0
PM10          0
NOx           0
O3            0
CO            0
HR            0
NO            0
NO2           0
TMP           0
dtype: int64

In [72]:
data_hourly.describe()

,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,TMP
count,5.848182e+06,5.848182e+06,5.848182e+06,5.848182e+06,5.848182e+06,5.848182e+06,5.848182e+06,5.848182e+06,5.848182e+06,5.848182e+06
mean,1.643294e+02,2.441384e+01,5.270074e+01,3.431426e-02,2.547988e-02,9.878910e-01,6.036240e+01,1.781668e-02,1.777698e-02,1.839740e+01
std,8.867297e+01,2.157374e+01,3.676655e+01,2.557446e-02,1.711418e-02,6.587089e-01,1.685173e+01,1.962701e-02,1.073153e-02,5.372382e+00
min,3.200000e+01,3.012900e+00,9.400056e+00,3.200100e-03,2.000081e-03,1.560028e-01,1.663104e+01,5.000100e-04,2.100028e-03,6.070100e+00
25%,1.020000e+02,1.450119e+01,3.105655e+01,1.864065e-02,1.200000e-02,5.350000e-01,5.154000e+01,6.000000e-03,1.000000e-02,1.529816e+01
50%,1.370000e+02,2.110023e+01,4.790000e+01,3.036875e-02,2.431242e-02,8.200000e-01,6.036240e+01,1.339321e-02,1.620000e-02,1.839740e+01
75%,2.450000e+02,2.723187e+01,5.900000e+01,4.083036e-02,3.200000e-02,1.202126e+00,7.000000e+01,2.163440e-02,2.300000e-02,2.188456e+01
max,4.260000e+02,3.018800e+02,3.270700e+02,2.069858e-01,9.399300e-02,4.166400e+00,9.999999e+01,1.729900e-01,6.339660e-02,3.361900e+01


สังเกตว่า ค่าทางสถิตไม่มีการเปลี่ยนแปลงใดๆ หลังเติมค่าเฉลี่ย

---  
ส่วนส่งออกเป็นไฟล์ csv เพื่อใช้งานต่อได้ง่าย

In [ ]:
data_hourly.to_csv("../data/data_hourly_clean_hourly.csv", index=False)

In [ ]:
data_daily = data_hourly.groupby([pd.Grouper(freq="1D", key="datetime"), "station_id"]).mean()

In [ ]:
data_daily.to_csv("../data/data_hourly_clean_daily.csv")